In [26]:
import sqlite3
from sqlite3 import Error

## Generic Python Libraries For SQL

### current code

The current code is using sqlite3 and psycopg2

## Turboodbc (recommended option due to its speed)

Turbodbc is a Python module to access relational databases via the Open Database Connectivity (ODBC) interface. Its primary target audience are data scientists that pull and insert huge amount of data from and to the database.

For maximum compatibility, turbodbc complies with the Python Database API Specification 2.0 (PEP 249). For maximum performance, turbodbc offers built-in NumPy and Apache Arrow support and internally relies on batched data transfer instead of single-record communication as other popular ODBC modules do.

[Docs](https://turbodbc.readthedocs.io/en/latest/index.html)

### Pyodbc

Pyodbc is an open-source Python module that makes accessing ODBC databases simple. It implements the DB API 2.0 specification but is packed with even more Pythonic convenience.

``` conda install pyodbc```

[docs](https://github.com/mkleehammer/pyodbc/wiki)

### SQLAlchemy

SQLAlchemy is a popular SQL toolkit and Object Relational Mapper. It's written in Python and it gives the full power and flexibility of SQL to an application developer. It is open-source and cross-platform software released under a MIT license. SQLAlchemy is famous for its object-relational mapper (ORM). Its classes can be mapped to the database, thereby allowing the object model and database schema to develop in a cleanly decoupled way right from the beginning. 

[docs](https://www.tutorialspoint.com/sqlalchemy/index.htm)

## Connecting to the database

In [27]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

# To execute queries in SQLite, use cursor.execute().
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")
        
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

.execute() can execute any query passed to it in the form of string. We'll use this method to create tables in this section. In the upcoming sections, We'll use this same method to execute update and delete queries as well.

In [28]:
connection = create_connection("sm_app.sqlite")

Connection to SQLite DB successful


The database file will be created automatically if it doesn't exists. If it does the code above will connect Python to existing database.

## Creating tables

We'll create four tables:

    * users
    * posts
    * comments
    * likes


In [29]:
# user table creation query
create_users_table = """
CREATE TABLE IF NOT EXISTS users (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  age INTEGER,
  gender TEXT,
  nationality TEXT
);
"""

In [30]:
execute_query(connection, create_users_table)  

Query executed successfully


In [31]:
# post table creation query
create_posts_table = """
CREATE TABLE IF NOT EXISTS posts(
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  title TEXT NOT NULL, 
  description TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id)
);
"""

In [32]:
execute_query(connection, create_posts_table)

Query executed successfully


In [33]:
create_comments_table = """
CREATE TABLE IF NOT EXISTS comments (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  text TEXT NOT NULL, 
  user_id INTEGER NOT NULL, 
  post_id INTEGER NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

create_likes_table = """
CREATE TABLE IF NOT EXISTS likes (
  id INTEGER PRIMARY KEY AUTOINCREMENT, 
  user_id INTEGER NOT NULL, 
  post_id integer NOT NULL, 
  FOREIGN KEY (user_id) REFERENCES users (id) FOREIGN KEY (post_id) REFERENCES posts (id)
);
"""

execute_query(connection, create_comments_table)  
execute_query(connection, create_likes_table) 

Query executed successfully
Query executed successfully


We can see that creating tables in SQLite is very similar to using raw SQL. All we have to do is store the query in a string variable and then pass that variable to cursor.execute().

## Inserting records

In [34]:
create_users = """
INSERT INTO
  users (name, age, gender, nationality)
VALUES
  ('James', 25, 'male', 'USA'),
  ('Leila', 32, 'female', 'France'),
  ('Brigitte', 35, 'female', 'England'),
  ('Mike', 40, 'male', 'Denmark'),
  ('Elizabeth', 21, 'female', 'Canada');
"""

execute_query(connection, create_users)   

Query executed successfully


In [35]:
create_posts = """
INSERT INTO
  posts (title, description, user_id) 
VALUES
  ("Happy", "I am feeling very happy today", 1),
  ("Hot Weather", "The weather is very hot today", 2),
  ("Help", "I need some help with my work", 2),
  ("Great News", "I am getting married", 1),
  ("Interesting Game", "It was a fantastic game of tennis", 5),
  ("Party", "Anyone up for a late-night party today?", 3);
"""

execute_query(connection, create_posts)  

Query executed successfully


In the previous table the user_id key is a foreign key and therefore must be specified

In [36]:
create_comments = """
INSERT INTO
  comments (text, user_id, post_id)
VALUES
  ('Count me in', 1, 6),
  ('What sort of help?', 5, 3),
  ('Congrats buddy', 2, 4),
  ('I was rooting for Nadal though', 4, 5),
  ('Help with your thesis?', 2, 3),
  ('Many congratulations', 5, 4);
"""

create_likes = """
INSERT INTO
  likes (user_id, post_id)
VALUES
  (1, 6),
  (2, 3),
  (1, 5),
  (5, 4),
  (2, 4),
  (4, 2),
  (3, 6);
"""

execute_query(connection, create_comments)
execute_query(connection, create_likes) 

Query executed successfully
Query executed successfully


Same a before the user_id and post_id are foreign keys and must be specified

## Selecting Records



To select records using SQLite, we can again use cursor.execute(). However, after you’ve done this, we'll need to call .fetchall(). This method returns a list of tuples where each tuple is mapped to the corresponding row in the retrieved records.def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as e:
        print(f"The error '{e}' occurred")

In [37]:
select_users = "SELECT * from users"
users = execute_read_query(connection, select_users)

for user in users:
    print(user)


(1, 'James', 25, 'male', 'USA')
(2, 'Leila', 32, 'female', 'France')
(3, 'Brigitte', 35, 'female', 'England')
(4, 'Mike', 40, 'male', 'Denmark')
(5, 'Elizabeth', 21, 'female', 'Canada')
(6, 'James', 25, 'male', 'USA')
(7, 'Leila', 32, 'female', 'France')
(8, 'Brigitte', 35, 'female', 'England')
(9, 'Mike', 40, 'male', 'Denmark')
(10, 'Elizabeth', 21, 'female', 'Canada')


It's not recommended to use SELECT * on large tables since it can result in a large number of I/O operations that increase the network traffic.
so using LIMIT 10 will be usefuyll

In [38]:
update_post_description = """
UPDATE
  posts
SET
  description = "The weather has become pleasant now"
WHERE
  id = 2
"""


In [39]:

execute_query(connection, update_post_description)

Query executed successfully


## Deleting Table Records

In [40]:
delete_comment = "DELETE FROM comments WHERE id = 5"
execute_query(connection, delete_comment)

Query executed successfully


We have seen how to work with SQLite databases from Python interface. sqlite3 is a special package that connects to any SQLite database. Other database types have other packages with pre-build drivers therefore we can't use sqlite3 to connect to those.
   * mysql-connector-python for MySQL
   * psycopg2 for Postgres
   * SQLAlchemy - generic library which using JDBC to connect to different DBs.
